In [265]:
import numpy as np
import nibabel as nib
from nibabel.affines import apply_affine
import matplotlib.pyplot as plt
import os

In [266]:
# Load data
# Load crop of occip ROI (cropped from rh 200um volume)
occipCrop = nib.load('raw_data/RHoccip_SGcrop_200um.nii.gz')
occipCropData = occipCrop.get_fdata()
print(f"RH Occip crop volume shape: {occipCropData.shape}")

RH Occip crop volume shape: (83, 92, 89)


In [267]:
# Slice from original (end exclusive)
x_start, x_end = 239, 329  # sagittal
y_start, y_end = 202, 292  # axial
z_start, z_end = 108, 198  # coronal

In [268]:
if False:

    # affine transforms
    # Full brain 100um affine matrix (assumed source of freesurfer surfaces but not sure)
    full_affine = np.array([[-0.1, 0., 0., 80.0],
                            [0., 0., 0.1, -78.05],
                            [0., -0.1, 0., 71.03],
                            [0., 0., 0., 1.]])

    # offset from full brain to RH crop in 100um voxel space
    offset = np.array([263, 50, 0])


In [269]:
if False: # cras from surface metadata

    coords, faces, volume_info = nib.freesurfer.read_geometry(
        'raw_data/FreeSurfer_Surfaces/rh.equi400.5.pial',
        read_metadata=True
    )

    # nibabel SurfaceRAS to voxel using metadata
    from nibabel.freesurfer import MGHImage

    # Extract the transformation info
    print("Volume info keys:", volume_info.keys())

    # surfaces metadata

    volume_shape = np.array(volume_info['volume'])  # [1760, 1280, 1760]
    voxelsize = np.array(volume_info['voxelsize'])  # [0.1, 0.1, 0.1]
    xras = np.array(volume_info['xras'])  # [-1, 0, 0]
    yras = np.array(volume_info['yras'])  # [0, 0, -1]
    zras = np.array(volume_info['zras'])  # [0, 1, 0]
    cras = np.array(volume_info['cras'])  # [-8.0, 9.94999695, 7.03147888]

    print(cras)



In [270]:
if False: # find valid vertex coords that fall into cropped volume

    all_voxel_coords = []  # voxel coordinates for each depth layer
    all_masks = []  # validity masks for each depth layer
    cras = np.array([-8.0, 9.95, 7.0315])

    for depth in depths:
        # filenames by depth
        surf_file = f'raw_data/FreeSurfer_Surfaces/rh.equi_test{depth:.1f}.pial'
        
        # Load surface vertex coordinates (RAS space, mm) and faces
        coords, faces = nib.freesurfer.read_geometry(surf_file)

        scanner_ras = coords + cras
        
        # Full brain RAS (mm) to Full brain voxels (100um)
        voxels_fullbrain = apply_affine(np.linalg.inv(full_affine), scanner_ras)
        
        # Apply offset: full brain voxels to RH crop voxels (100um)
        voxels_rh_100um = voxels_fullbrain - offset
        
        # downsample RH crop voxels (100um) to RH crop voxels (200um)
        voxels_rh_200um = voxels_rh_100um / 2.0
        
        # Check which vertices fall within the RH 200um volume bounds
        inCrop = ((voxels_rh_200um[:, 0] >= 0) & (voxels_rh_200um[:, 0] < rhOccipData.shape[0]) &
                (voxels_rh_200um[:, 1] >= 0) & (voxels_rh_200um[:, 1] < rhOccipData.shape[1]) &
                (voxels_rh_200um[:, 2] >= 0) & (voxels_rh_200um[:, 2] < rhOccipData.shape[2]))
        
        # Store results for this depth layer
        all_masks.append(inCrop)
        all_voxel_coords.append(voxels_rh_200um)
        
        print(f"Depth {depth:.2f}: {inCrop.sum()} vertices in bounds")

    # boolean masking array
    all_masks = np.array(all_masks)  # Shape: (21, N_vertices)
    all_voxel_coords = np.array(all_voxel_coords)  # Shape: (21, N_vertices, 3)




In [ ]:
# filter for smaller crop
import os
surf_dir = 'raw_data/FreeSurfer_Surfaces/20_depths'
surf_files = sorted([f for f in os.listdir(surf_dir) if f.startswith('rh.equi')])
depths = len(surf_files)
print(f"{depths} surface files")
#print(surf_files)
# Get cras offset from metadata. Surface vertices are in surfaceRAS/TkReg RAS and need to be mapped to scannerRAS. scannerRAS = surfRAS + cras
coords, __ , volume_info = nib.freesurfer.read_geometry('raw_data/FreeSurfer_Surfaces/20_depths/rh.equi0.5.pial', read_metadata=True)
cras = np.array(volume_info['cras'])
print(f"filename: {surf_files[10]}, with cras: {cras}")


all_voxel_coords = []
all_masks = []
for depth_idx, surf_filename in enumerate(surf_files):
    surf_file = os.path.join(surf_dir, surf_filename)
    #print(f"Depth {depth_idx}: {surf_filename}")
    coords, faces = nib.freesurfer.read_geometry(surf_file)
    scanner_ras = coords + cras
    voxels_all = apply_affine(np.linalg.inv(occipCrop.affine), scanner_ras)
    
    # only keep voxels within crop bounds
    inCrop = ((voxels_all[:, 0] >= 0) & (voxels_all[:, 0] < occipCrop.shape[0]) &
              (voxels_all[:, 1] >= 0) & (voxels_all[:, 1] < occipCrop.shape[1]) &
              (voxels_all[:, 2] >= 0) & (voxels_all[:, 2] < occipCrop.shape[2]))
    
    
    all_masks.append(inCrop)
    all_voxel_coords.append(voxels_all)

all_masks = np.array(all_masks)
all_voxel_coords = np.array(all_voxel_coords)

21 surface files
filename: rh.equi0.5.pial, with cras: [-8.          9.94999695  7.03147888]


In [272]:
# filter only full sets of depth
depth_start = 2
depth_end = depths -2

# vertices that are within bounds across all depth layers
valid_layers = np.all(all_masks[depth_start:depth_end,:], axis=0)  # axis=0 check across depth dimension

print(len(valid_layers))
n_valid = valid_layers.sum()
n_total = len(valid_layers)
print(f"\nVertices valid all {depths} layers: {n_valid} out of {n_total} ({100*n_valid/n_total:.1f}%)")

249329

Vertices valid all 21 layers: 3382 out of 249329 (1.4%)


In [273]:
# extract intensity profiles

# array to store intensity profiles
# Rows = valid vertices, Columns = depth layers
intensity_profiles = np.zeros((n_valid, (depth_end-depth_start)))

# Extract voxel coordinates for valid vertices only
valid_coords = all_voxel_coords[depth_start:depth_end, valid_layers, :]  # Shape: (21, n_valid, 3)

# Loop through each depth layer to sample intensities
for depth_idx in range(0,(depth_end - depth_start)):
    # Get voxel coordinates for this depth layer
    voxels = valid_coords[depth_idx]  # Shape: (n_valid, 3)
    
    # Convert continuous coordinates to integer voxel indices
    voxel_indices = np.floor(voxels).astype(int)
    
    # Sample intensities from volume at these voxel locations
    intensities = occipCropData[voxel_indices[:, 0], voxel_indices[:, 1], voxel_indices[:, 2]]
    
    # Store in profile array\
    intensity_profiles[:, depth_idx] = intensities

print(f"extracted profiles with shape: {intensity_profiles.shape}")
print(f"Intensity range: {intensity_profiles.min():.2f} to {intensity_profiles.max():.2f}")

extracted profiles with shape: (3382, 17)
Intensity range: 166.19 to 3343.38


In [274]:
import ipywidgets as widgets
from IPython.display import display
# No. of profiles to display
n_profiles_per_view = 10

# interactive plotting
def plot_profiles(start_index):
    """
    Plot batch of cortical depth profiles starting from start_index
    """
    # Calculate end index for this batch
    end_index = min(start_index + n_profiles_per_view, n_valid)
    
    # Select depth range to plot
    depths_to_plot = np.linspace(0,1,depths)
    depth_start_idx = 0
    depth_end_idx = 17  
    depths_to_plot = depths_to_plot[depth_start_idx:depth_end_idx]
    
    
    plt.figure(figsize=(12, 8))
    
    # Plot each profile in the current batch
    for idx in range(start_index, end_index):
        profile = intensity_profiles[idx, depth_start_idx:depth_end_idx]
        plt.plot(depths_to_plot, profile, alpha=0.6, linewidth=1)
    
    # plot stuff
    plt.xlabel('Cortical depth (0 = pial, 1 = WM)', fontsize=10)
    plt.ylabel('Intensity (T2*)', fontsize=10)
    plt.title(f'Cortical Depth Intensity Profiles (Vertices {start_index} to {end_index-1} depths {depth_start_idx}:{depth_end_idx})', fontsize=12)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()

In [275]:
# Create slider widget
# range 0 to (n_valid - n_profiles_per_view) in steps of n_profiles_per_view
widgets.interact(
    plot_profiles,
    start_index=widgets.IntSlider(
        value=0,  # Initial position
        min=0,  # Start at vertex 0
        max=n_valid - n_profiles_per_view,  # End when less than 10 profiles remain
        step=n_profiles_per_view,  # stepsize for slider
        description='profiles:',  # slider label
        continuous_update=False  
    )
)

# summary stats
print(f"Total valid vertices: {n_valid}")
print(f"Profiles per view: {n_profiles_per_view}")
print(f"Mean intensity across all profiles: {intensity_profiles.mean():.2f}")
print(f"Std intensity across all profiles: {intensity_profiles.std():.2f}")

interactive(children=(IntSlider(value=0, continuous_update=False, description='profiles:', max=3372, step=10),…

Total valid vertices: 3382
Profiles per view: 10
Mean intensity across all profiles: 2155.78
Std intensity across all profiles: 324.15


In [276]:
if False:

    # random vertex for visualisation checking
    test_vertex_id = 6349

    # Check if vertex is in the valid set
    if valid_all_layers[test_vertex_id]:
        print(f"Vertex {test_vertex_id} is valid across all layers")
        
        # Get coordinates for this vertex at all depths
        test_coords = all_voxel_coords[:, test_vertex_id, :]  # Shape: (depths, 3)
        
        # Sample intensities for depth range
        for depth_idx in range(8, depths - 8):
            # Get coordinate at this depth
            coord = test_coords[depth_idx]  # Shape: (3,) = [x, y, z]
            
            # Convert to integer voxel index
            voxel_idx = np.floor(coord).astype(int)
            
            # Sample intensity
            intensity = occipCropData[voxel_idx[0], voxel_idx[1], voxel_idx[2]]
            
            print(f"Depth {depth_idx}: voxel {voxel_idx}, intensity {intensity:.2f}")
    else:
        print(f"Vertex {test_vertex_id} is not valid - outside crop at some depth")


In [281]:
def check_vertex_profile(vertex_id, depth_start=2, depth_end=None):
    """Check intensity profile for a specific vertex"""
    if depth_end is None:
        depth_end = depths - 2
    
    if not valid_layers[vertex_id]:
        print(f"Vertex {vertex_id} invalid (outside crop)")
        return
    
    coords = all_voxel_coords[:, vertex_id, :]
    
    print(f"Vertex {vertex_id} profile (depths {depth_start}-{depth_end}):\n")
    for d in range(depth_start, depth_end):
        voxel = np.floor(coords[d]).astype(int)
        intensity = occipCropData[voxel[0], voxel[1], voxel[2]]
        print(f"  Depth {d}: voxel {voxel}, intensity {intensity:.2f}")

check_vertex_profile(8789, 8, 21)       # Custom range

Vertex 8789 profile (depths 8-21):

  Depth 8: voxel [51 41 34], intensity 2386.41
  Depth 9: voxel [51 41 33], intensity 2113.62
  Depth 10: voxel [50 41 33], intensity 2189.44
  Depth 11: voxel [50 42 33], intensity 2201.64
  Depth 12: voxel [50 42 32], intensity 1804.69
  Depth 13: voxel [50 42 32], intensity 1804.69
  Depth 14: voxel [50 43 32], intensity 2130.24
  Depth 15: voxel [50 43 31], intensity 2108.73
  Depth 16: voxel [49 44 31], intensity 1909.55
  Depth 17: voxel [49 44 31], intensity 1909.55
  Depth 18: voxel [49 44 30], intensity 1947.82
  Depth 19: voxel [49 45 30], intensity 1989.71
  Depth 20: voxel [49 45 29], intensity 1495.12


In [ ]:
if False:
    coords, faces = nib.freesurfer.read_geometry('raw_data/FreeSurfer_Surfaces/rh.equi_test0.5.pial')
    # Load FreeSurfer surface

    nib.freesurfer.write_geometry('raw_data/FreeSurfer_Surfaces/rh.equi_test0.5.vtk', 
                                coords, faces)
    # Save as VTK format